In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import pandas as pd 
import numpy as np 
import math

In [12]:
dfdata = pd.read_csv('/content/drive/MyDrive/TH1391_NLMH/Week9_Thuchanh3/computer.csv')
dfdata

,Age,Income,Student,Credit_rating,Buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


In [13]:
class Tree:
  def __init__(self,observationIDs,features,currLvl=0,subTree={},bestFeature=None,majorityLabel=None,parentMajorityLabel=None):
    self.observationIDs = observationIDs
    self.features = features
    self.currLvl = currLvl
    self.subTree = subTree
    self.bestFeature = bestFeature
    self.majorityLabel = majorityLabel
    self.parentMajorityLabel = parentMajorityLabel
    self.setBestFeatureID(bestFeature)
    
  def setBestFeatureID(self, feature):
    idx = None
    if feature == 'Age':
      idx = 0
    elif feature == 'Income':
      idx = 1
    elif feature == 'Student':
      idx = 2
    else:
      idx = 3
    self.bestFeatureID = int(idx)

In [14]:
def predict(tree, obs):
	if tree.bestFeature == None:
		return tree.majorityLabel
	featVal = obs[tree.bestFeatureID]
	if not featVal in tree.subTree: # val with no subtree
		return tree.majorityLabel
	else: # recurse on subtree
		return predict(tree.subTree[featVal],obs)

In [15]:
def displayDecisionTree(tree):
	print('\t'*tree.currLvl + '(lvl {}) {}'.format(tree.currLvl,tree.majorityLabel))
	if tree.bestFeature == None:
		return

	print('\t'*tree.currLvl + '{}'.format(tree.bestFeature) + ': ')
	for [val,subTree] in sorted(tree.subTree.items()):
		print('\t'*(tree.currLvl+1) + 'choice: {}'.format(val))
		displayDecisionTree(subTree)

In [16]:
def Entropy(ns):
	entropy = 0.0
	total = sum(ns)
	for x in ns:
		entropy += -1.0*x/total*math.log(1.0*x/total,2)
	return entropy

In [17]:
def IG(observationIDs, feature,dfdata):
	# get smaller dataframe
	df = dfdata.loc[list(observationIDs)]
	# populate counts for Wins/Losses for each category of the feature
	labelCountDict = {}
	valueLabelCountDict = {}
	for index, row in df.iterrows():
		label = row['Buys_computer']
		if not label in labelCountDict:
			labelCountDict[label] = 0 # this specific label was not found so insert 0 count
		labelCountDict[label] += 1
		featureValue = row[feature]
		if not featureValue in valueLabelCountDict:
			valueLabelCountDict[featureValue] = {} # this specific feature value not found so insert empty dict
		if not label in valueLabelCountDict[featureValue]:
			valueLabelCountDict[featureValue][label] = 0 # this specific label was not found for this feature value so insert 0 count
		valueLabelCountDict[featureValue][label] += 1

	ns = []
	for [label,count] in labelCountDict.items():
		ns.append(count)

	H_Y = Entropy(ns)

	H_Y_X = 0.0
	for [featureValue, labelCountDict] in valueLabelCountDict.items():
		nsHYX = []
		for [label,count] in labelCountDict.items():
			nsHYX.append(count)
		H_Y_X += 1.0*sum(nsHYX)/len(df)*Entropy(nsHYX)
	return H_Y - H_Y_X

In [18]:
def GR(observationIDs, feature,dfdata):
	ig = IG(observationIDs,feature,dfdata)
	if ig == 0:
		return 0
	df = dfdata.loc[list(observationIDs)]
	valueLabelDict = {}
	for index, row in df.iterrows():
		label = row['Buys_computer']
		featureValue = row[feature]
		if featureValue not in valueLabelDict:
			valueLabelDict[featureValue] = 0
		valueLabelDict[featureValue] += 1
	ns = []
	for [val,count] in valueLabelDict.items():
		ns.append(count)
	ent = Entropy(ns)
	return float(ig)/ent

In [19]:
def fillDecisionTree(tree,decisionTreeAlgo,dfdata):
	# find the majorityLabel
	df = dfdata.loc[list(tree.observationIDs)] # smaller df
	counts = df['Buys_computer'].value_counts()
	majorityLabel = df['Buys_computer'].value_counts().idxmax()
	if len(counts) > 1:
		if counts['yes'] == counts['no']:
			majorityLabel = tree.parentMajorityLabel
	tree.majorityLabel = majorityLabel

	# exit if only one label
	if len(counts) == 1:
		return
	# exit if no features left
	if len(tree.features) == 0: 
		return

	# find best feature
	featureValueDict = {}
	for feature in tree.features: 
		if decisionTreeAlgo == 'ID3':
			metricScore = IG(tree.observationIDs,feature,dfdata)
		if decisionTreeAlgo == 'C45':
			metricScore = GR(tree.observationIDs,feature,dfdata)
		featureValueDict[feature] = metricScore
	bestFeature, bestFeatureValue = sorted(featureValueDict.items(),reverse=True)[0]
	# exit if IG or GR is 0
	if bestFeatureValue == 0.0:
		return
	tree.bestFeature = bestFeature

	# find subset of features
	subFeatures = set()
	for feature in tree.features:
		if feature == bestFeature: # skip the current best feature
			continue
		subFeatures.add(feature)
	
	# find best feature id
	bestFeatureIdx = 0
	if bestFeature == 'Age':
		bestFeatureIdx = 0
	elif bestFeature == 'Income':
		bestFeatureIdx = 1
	elif bestFeature == 'Student':
		bestFeatureIdx = 2
	else:
		bestFeatureIdx = 3
	
	# find subset of observations
	subObservationsDict = {}
	for obs in tree.observationIDs:
		val = dfdata.values[obs][bestFeatureIdx]
		if not val in subObservationsDict:
			subObservationsDict[val] = set()
		subObservationsDict[val].add(obs)

	for [val,obs] in subObservationsDict.items():

		tree.subTree[val] = Tree(obs, subFeatures, tree.currLvl + 1,{},None,None,majorityLabel)
		
		fillDecisionTree(tree.subTree[val],decisionTreeAlgo,dfdata)

In [20]:
initialObservationIDs = set(range(len(dfdata)))
initialFeatures = set(dfdata.columns.values[:-1])

In [21]:
algoChoice = str(input(("Which decision tree algorithm would you like to use ('ID3' or 'C45)?")))
if algoChoice not in {'ID3','C45'}:
	print("Invalid algorithm choice. You must choose 'ID3' or 'C45'")
	exit()

print("choice: {}".format(algoChoice))

Which decision tree algorithm would you like to use ('ID3' or 'C45)?ID3
choice: ID3


In [22]:
MyTree = Tree(initialObservationIDs,initialFeatures)
fillDecisionTree(MyTree,algoChoice,dfdata)

print('My Decision Tree:')
displayDecisionTree(MyTree)

My Decision Tree:
(lvl 0) yes
Student: 
	choice: no
	(lvl 1) no
	Income: 
		choice: high
		(lvl 2) no
		Credit_rating: 
			choice: excellent
			(lvl 3) no
			choice: fair
			(lvl 3) no
			Age: 
				choice: middle_aged
				(lvl 4) yes
				choice: youth
				(lvl 4) no
		choice: medium
		(lvl 2) no
	choice: yes
	(lvl 1) yes
	Income: 
		choice: high
		(lvl 2) yes
		choice: low
		(lvl 2) yes
		Credit_rating: 
			choice: excellent
			(lvl 3) yes
			Age: 
				choice: middle_aged
				(lvl 4) yes
				choice: senior
				(lvl 4) no
			choice: fair
			(lvl 3) yes
		choice: medium
		(lvl 2) yes


In [23]:
obs=['middle_aged','high','no','fair']
predict(MyTree,obs)

'yes'

In [24]:
data_test = dfdata.iloc[:,:-1]
data_test

,Age,Income,Student,Credit_rating
0,youth,high,no,fair
1,youth,high,no,excellent
2,middle_aged,high,no,fair
3,senior,medium,no,fair
4,senior,low,yes,fair
5,senior,low,yes,excellent
6,middle_aged,low,yes,excellent
7,youth,medium,no,fair
8,youth,low,yes,fair
9,senior,medium,yes,fair


In [25]:
for index, row in data_test.iterrows():
    obs_row = [row[0], row[1], row[2], row[3]]
    pred = predict(MyTree, obs_row)
    print(f'Prediction for {obs_row}: {pred}')

Prediction for ['youth', 'high', 'no', 'fair']: yes
Prediction for ['youth', 'high', 'no', 'excellent']: yes
Prediction for ['middle_aged', 'high', 'no', 'fair']: yes
Prediction for ['senior', 'medium', 'no', 'fair']: yes
Prediction for ['senior', 'low', 'yes', 'fair']: yes
Prediction for ['senior', 'low', 'yes', 'excellent']: yes
Prediction for ['middle_aged', 'low', 'yes', 'excellent']: yes
Prediction for ['youth', 'medium', 'no', 'fair']: yes
Prediction for ['youth', 'low', 'yes', 'fair']: yes
Prediction for ['senior', 'medium', 'yes', 'fair']: yes
Prediction for ['youth', 'medium', 'yes', 'excellent']: yes
Prediction for ['middle_aged', 'medium', 'no', 'excellent']: yes
Prediction for ['middle_aged', 'high', 'yes', 'fair']: yes
Prediction for ['senior', 'medium', 'no', 'excellent']: yes
